In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

file_path =  "/content/gdrive/My Drive/Colab Notebooks/wtm/finalproj/"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
import re
from urllib import request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd

In [ ]:
# 헌법재판소 웹사이트

url_f = 'https://www.ccourt.go.kr/site/kor/ex/bbs/List.do?cbIdx=1106&pageIndex=' # 크롤링할 헌재 사이트 몸통 앞 부분 (뒤에 페이지 수 붙을 거임)
site = 'https://www.ccourt.go.kr' #판결문 요약으로 들어가는 사이트 앞 주소
rec = "/site/kor/ex/bbs/View.do?cbIdx" #반복되는 부분 뒷부분

# 헌법재판소 분야별 주요 판례 Scraping

In [ ]:
def Search(getA):

  global hunjea_cases
  global index

  for getLink in getA:
       data = getLink.get('href')
       #print(data)

       try:
           if rec in getLink.get("href"):
                   accessUrl = site + getLink.get("href") # 개별 판결문 게시글 링크 저장
                   if index % 10 == 1 : print(f'\n💖 {index}번째 게시글 링크 : ', accessUrl)

                   r = requests.get(accessUrl)  #개별 판결문 게시글 링크로 접속
                   soup = BeautifulSoup(r.text, "html.parser")

                   CaseNum = soup.select('span.caseNum')[0].text
                   if index % 10 == 1 : print('사건번호 = ', CaseNum)

                   subTitle = soup.select('span.nickname')[0].text
                   if index % 10 == 1 : print('사건명 = ', subTitle[5:])

                   category = soup.select('span.category')[0].text
                   if index % 10 == 1 : print('분야 = ', category)

                   Content = soup.select('div.viewCon')[0].text.replace('\n', " ").replace('\t', " ").replace('\r', " ") #개별 판결문 contents
                   if index % 10 == 1 : print('판례 본문 = ', Content)

                   hunjea_cases.loc[index] = [CaseNum, subTitle[5:], category, Content]
                   index += 1


       except: pass

  print(index)
  return hunjea_cases


# **Main**

In [ ]:
global index
index = 1
global hunjea_cases
hunjea_cases = pd.DataFrame(columns=("CaseNum", "Title", "Category", "Content"))


maxpage = input("🟡 검색 할 페이지수 (최대 66p): ")
maxpage = int(maxpage)

for page in range(1, maxpage+1) : #스크래핑 하고 싶은 영상의 총 page 수 = 66 (hyperparam)

  start = time.time()    # 시작 시간 기록

  url = url_f + str(page)

  print('▶ 수집 중인 페이지 = ', page)
  # request 모듈을 사용하여 웹 페이지의 내용을 가져온다
  r = requests.get(url)
  print("헌법재판소 site 접속 여부: ", r)

  # beautiful soup 초기화
  soup = BeautifulSoup(r.text, "html.parser")

  # 태그로 찾기 (모든 항목)
  getA = soup.find_all("a")
  Search(getA)
  #print(getA)

  print(f'{page} page Completed! 🌞🌞🌞🌞🌞🌞🌞🌞 ')

  processing_time = time.time() - start    # 경과 시간 계산
  times = str(datetime.timedelta(seconds = processing_time))
  print()

print("◎ 수행시간 = ", times)

🟡 검색 할 페이지수 (최대 66p): 66
▶ 수집 중인 페이지 =  1
헌법재판소 site 접속 여부:  <Response [200]>

💖 1번째 게시글 링크 :  https://www.ccourt.go.kr/site/kor/ex/bbs/View.do?cbIdx=1106&bcIdx=1007778
사건번호 =  2020헌마542
사건명 =  주민등록법상 지문날인제도 관련 사건
분야 =  언론 등 정신적 자유에 관한 결정
기사 본문 =   주민등록법상 지문날인제도 사건<헌재 2024. 4. 25. 2020헌마542  주민등록법 제24조 제2항 위헌확인 등> 이 사건은 주민등록증 발급신청서에 열 손가락 지문을 날인하도록 하고, 경찰청장이 주민등록증 발급신청서에 날인된 지문정보를 수집, 보관하여 범죄수사목적에 이용하는 행위가 개인정보자기결정권을 침해하지 않는다고 결정한 사안이다.     【사건의 배경】2003년에 태어난 청구인1은 2020년 2월 주민등록증 발급통지서를 받았으나 주민등록증 발급신청서에 열 손가락의 지문날인을 거부하며 주민등록증 발급신청을 하지 않았다. 1985년에 태어난 청구인2는 2002년 주민등록증을 발급받았다.청구인1은 주민등록증에 지문을 수록하도록 한 주민등록법 제24조 제2항 본문 중 ‘지문’에 관한 부분, 주민등록증의 발급을 위해서는 주민등록증 발급신청서에 좌우 열 손가락의 회전지문과 평면지문을 날인하도록 한 주민등록법 시행령 제36조 제3항에 의한 별지 제30호 서식 중 열 손가락의 지문을 찍도록 한 부분 및 시장·군수·구청장으로 하여금 주민등록증 발급신청서를 해당자의 주민등록지를 관할하는 경찰서의 지구대장 또는 파출소장에게 보내도록 한 주민등록법 시행규칙 제8조가 자신의 기본권을 침해한다고 주장하며, 청구인2는 피청구인 경찰청장이 주민등록증 발급신청서에 날인되어 있는 자신의 지문정보를 보관·전산화하고 이를 범죄수사목적에 이용하는 행위가 자신의 기본권을 침해한다고 주장하며 헌법소원심판을 청구하였다.    【심판대상조항】이 사건 심판대

In [ ]:
hunjea_cases.shape

(648, 4)

In [ ]:
hunjea_cases.head(10)

,CaseNum,Title,Category,Content
1,2020헌마542,주민등록법상 지문날인제도 관련 사건,언론 등 정신적 자유에 관한 결정,주민등록법상 지문날인제도 사건<헌재 2024. 4. 25. 2020헌마542 주...
2,2020헌가4,유류분에 관한 위헌제청 및 헌법소원 사건,가족 · 노동 등 사회관계에 관한 결정,민법상 유류분제도 사건 <헌재 2024. 4. 25. 2020헌가4 등 민...
3,2019헌마500,주 52시간 상한제 사건,가족 · 노동 등 사회관계에 관한 결정,주 52시간 상한제 사건 <헌재 2024. 2. 28. 2019헌마500 최저임금...
4,2022헌마356,태아의 성별 고지 제한 사건,가족 · 노동 등 사회관계에 관한 결정,태아의 성별고지 금지 위헌확인 사건 <헌재 2024. 2. 28. 2022헌마35...
5,2020헌마1476,동의 없는 가명정보 처리 사건,언론 등 정신적 자유에 관한 결정,동의 없는 가명정보 처리 사건 <헌재 2023. 10. 26. 2020헌마...
6,2017헌가16,동성 군인 간의 추행 사건,절차적 기본권 및 형사관계에 관한 결정,"동성 군인 간의 추행 사건 <헌재 2023. 10. 26. 2017헌가16, 20..."
7,2019헌마1165,"외국인 국민건강보험 지역가입자의 보험료 하한 산정기준, 세대구성, 보험료 체납정보 ...",가족 · 노동 등 사회관계에 관한 결정,외국인에 대한 국민건강보험 제도 차별 사건 <헌재 2023. 9. 26. 2019...
8,2017헌바42,국가보안법상 이적행위 및 이적표현물 제작행위 등 처벌규정에 관한 사건,언론 등 정신적 자유에 관한 결정,국가보안법상 이적행위 및 이적표현물 제작행위 등 처벌규정에 관한 사건 <...
9,2023헌나1,행정안전부장관(이상민) 탄핵,정치 · 선거관계에 관한 결정,행정안전부장관 탄핵 사건 <헌재 2023. 7. 25. 2023헌나1 행정안전부장...
10,2020헌마104,출판문화산업 진흥법 제22조 제4항 등 위헌확인,경제 · 재산권 · 조세관계에 관한 결정,도서정가제 사건 <헌재 2023. 7. 20. 2020헌마104 출판문화산...


In [ ]:
hunjea_cases.tail(5)

,CaseNum,Title,Category,Content
644,88헌가5등,보호감호 사건,절차적 기본권 및 형사관계에 관한 결정,가. 사건의 배경 이 사건은 제5공화국 당시 국가보위입법회의에서 제정되었던 ...
645,88헌가7,소송촉진특례법 사건,경제 · 재산권 · 조세관계에 관한 결정,가. 사건의 개요 이 사건은 헌법재판소가 창립된 이후 첫 번째 전원합의체 ...
646,대법원 70다1010,국가배상법 제2조 제1항 단서와 법원조직법 제59조 제1항 위헌결정,헌법위원회 및 대법원 헌법판례,원고 피상고인 김희원외 3명 피고 상 고 인 대한민국 제1심 서울민사지방 ...
647,1952.9.9 결정,농지개혁법 제18조 제1항 후단 및 제25조 제1항 후단 위헌결정,헌법위원회 및 대법원 헌법판례,제청법원 대법원 단기4285년 헌위 제1호 법률의 위헌여부결정제청사건에 관하...
648,1952.9.9 결정,비상사태하의범죄처벌에관한특별조치령 제9조 제1항 위헌결정,헌법위원회 및 대법원 헌법판례,제청법원 서울고등법원 단기4285년 헌위제2호 법률의 위헌여부결정 제청사건에...


In [ ]:
print(hunjea_cases['Category'].unique())
print()

print("◎ 총 분야 매체 수: ", len(hunjea_cases['Category'].unique()))

['언론 등 정신적 자유에 관한 결정' '가족 · 노동 등 사회관계에 관한 결정' '절차적 기본권 및 형사관계에 관한 결정'
 '정치 · 선거관계에 관한 결정' '경제 · 재산권 · 조세관계에 관한 결정' '헌법위원회 및 대법원 헌법판례']

◎ 총 분야 매체 수:  6


# 수집한 데이터를 csv file로 저장

In [ ]:
# 수집한 데이터를 파일로 저장

file_name = "분야별_헌재판례.csv"

file_full_path = file_path + file_name

hunjea_cases.to_csv(file_full_path , index = False, encoding="utf-8-sig") #sig!!!!!!!!!!!!!!!!

#encoding = utf-8로 하면 코랩에서는 잘 읽어져도 파일을 엑셀로 열면 깨짐.

In [ ]:
# 저장된 데이터 파일을 읽어 오기 (내용 확인 하기)

report = pd.read_csv(file_full_path)


report.shape

(648, 4)